In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
from io import BytesIO
import ipywidgets as widgets

# Move to the dataset directory
%cd /content/drive/My Drive/dataset/

# Set the base path and verify that it's pointing to the correct directory
base_path = os.getcwd()  # Corrected to os.getcwd() to retrieve the current directory path
train_dir = os.path.join(base_path, 'train')
val_dir = os.path.join(base_path, 'val')
test_dir = os.path.join(base_path, 'test')

print("Base Path:", base_path)
print("Train Directory:", train_dir)
print("Validation Directory:", val_dir)
print("Test Directory:", test_dir)



/content/drive/My Drive/dataset
Base Path: /content/drive/My Drive/dataset
Train Directory: /content/drive/My Drive/dataset/train
Validation Directory: /content/drive/My Drive/dataset/val
Test Directory: /content/drive/My Drive/dataset/test


**PEMBUATAN MODEL**

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(150, 150, 3)), # Gambar RGB 150x150
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
    ])

model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 74, 74, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 16)        2320      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 36, 36, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 32)       

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
    )

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
    )

# Optimize the datasets for training
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       )

validation_dataset_final = (validation_dataset
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE)
                            )

Found 8000 files belonging to 50 classes.
Found 2000 files belonging to 50 classes.


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_dataset_final,
    epochs=20,
    validation_data = validation_dataset_final,
    verbose = 1
)

Epoch 1/20
250/250 [==============================] - 34s 79ms/step - loss: 3.6450 - accuracy: 0.0339 - val_loss: 3.4291 - val_accuracy: 0.0530
Epoch 2/20
250/250 [==============================] - 16s 66ms/step - loss: 3.2607 - accuracy: 0.1175 - val_loss: 2.4647 - val_accuracy: 0.3530
Epoch 3/20
250/250 [==============================] - 17s 66ms/step - loss: 2.0792 - accuracy: 0.4313 - val_loss: 1.3893 - val_accuracy: 0.6355
Epoch 4/20
250/250 [==============================] - 17s 66ms/step - loss: 1.4465 - accuracy: 0.6037 - val_loss: 1.1424 - val_accuracy: 0.6965
Epoch 5/20
250/250 [==============================] - 17s 66ms/step - loss: 1.1756 - accuracy: 0.6727 - val_loss: 0.9227 - val_accuracy: 0.7490
Epoch 6/20
250/250 [==============================] - 16s 65ms/step - loss: 1.0052 - accuracy: 0.7175 - val_loss: 0.7857 - val_accuracy: 0.7885
Epoch 7/20
250/250 [==============================] - 17s 67ms/step - loss: 0.8965 - accuracy: 0.7460 - val_loss: 0.7854 - val_accuracy:

In [ ]:
uploader = widgets.FileUpload(accept="image/*", multiple=True)
display(uploader)
out = widgets.Output()
display(out)

# Function to make predictions
def file_predict(filename, file, out):
    """A function for creating the prediction and printing the output."""
    image = tf.keras.utils.load_img(file, target_size=(150, 150))
    image = tf.keras.utils.img_to_array(image)
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image, verbose=0)[0]

    # Sort and get the top 3 predictions
    top_indices = np.argsort(prediction)[-3:][::-1]  # Get indices of top 3 classes
    top_percentages = prediction[top_indices] * 100  # Convert probabilities to percentages

    classes = [
        'YA', 'WA', 'ULU', 'TEDONG', 'TALENG', 'TA', 'SUKU', 'SA', 'RA', 'PA', 'NYA', 'NGA', 'NA', 'MA', 'LA',
        'KA', 'JA', 'GEMPELAN-SA-SAPA', 'GEMPELAN-PA', 'GANTUNGAN-TA', 'GANTUNGAN-NYA', 'GANTUNGAN-NGA',
        'GANTUNGAN-NA', 'GANTUNGAN-MA', 'GANTUNGAN-LA', 'GANTUNGAN-KA', 'GANTUNGAN-JA', 'GANTUNGAN-GA',
        'GANTUNGAN-DA', 'GANTUNGAN-CA', 'GANTUNGAN-BA', 'GANTUNGAN-A', 'GA', 'DA', 'CECEK', 'CA', 'BISAH', 'BA',
        'ADEG-ADEG', 'A', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0'
    ]

    # Get class names and percentages
    top_classes = [classes[i] for i in top_indices]

    with out:
        # print(f'\nModel output: {prediction}')
        print(f'\nTop 3 predictions for {filename}:')
        for percentage, class_name in zip(top_percentages, top_classes):
            print(f'{percentage:.2f}% - {class_name}')

# Function to handle file uploads
def on_upload_change(change):
    """A function to handle uploaded files and run predictions."""
    for filename, file_info in change.new.items():  # Access filename and file data
        file_content = BytesIO(file_info['content'])  # Get file content as BytesIO
        file_predict(filename, file_content, out)

# Attach the observer
uploader.observe(on_upload_change, names='value')

FileUpload(value={}, accept='image/*', description='Upload', multiple=True)

Output()